In [1]:
import numpy as np

import strawberryfields as sf
from strawberryfields import ops
from strawberryfields import RemoteEngine


2022-11-23 16:16:44.605936: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import xcc.commands
xcc.commands.ping()


'Successfully connected to the Xanadu Cloud.'

In [3]:
from strawberryfields.utils import random_interferometer
U = random_interferometer(4)
np.set_printoptions(precision=16)
print(U)


[[-0.1787225407044426-0.6303491559482155j
   0.0009285822611827-0.2098399400496477j
  -0.0454846739599724+0.4539393228756476j
   0.3894951753184049+0.4084706553965713j]
 [-0.4839632276603804+0.4421884609536221j
   0.0459197148651991+0.2051236918340788j
  -0.5628458485272879+0.3834772916974102j
  -0.1311791188995151+0.2121470396207172j]
 [-0.1522847124840133+0.1387621689443624j
  -0.3944667960609368+0.0651474161204435j
   0.4232156706525905-0.203583030244661j
  -0.2680526529525196+0.7108420421917672j]
 [ 0.3023962336375494-0.0841314388732129j
  -0.796477216306217 +0.3428670194307852j
  -0.1553214129493898+0.2886826245548124j
   0.0446169942063203-0.2002288554485604j]]


In [4]:
prog = sf.Program(8, name="remote_job1")


In [5]:
with prog.context as q:
    # Initial squeezed states
    # Allowed values are r=1.0 or r=0.0
    ops.S2gate(1.0) | (q[0], q[4])
    ops.S2gate(1.0) | (q[1], q[5])
    ops.S2gate(1.0) | (q[3], q[7])

    # Interferometer on the signal modes (0-3)
    ops.Interferometer(U) | (q[0], q[1], q[2], q[3])
    ops.BSgate(0.543, 0.123) | (q[2], q[0])
    ops.Rgate(0.453) | q[1]
    ops.MZgate(0.65, -0.54) | (q[2], q[3])

    # *Same* interferometer on the idler modes (4-7)
    ops.Interferometer(U) | (q[4], q[5], q[6], q[7])
    ops.BSgate(0.543, 0.123) | (q[6], q[4])
    ops.Rgate(0.453) | q[5]
    ops.MZgate(0.65, -0.54) | (q[6], q[7])

    ops.MeasureFock() | q

In [6]:
eng = RemoteEngine("X8")


In [7]:
results = eng.run(prog, shots=20)

print(results.samples)

print(np.mean(results.samples, axis=0))

[[1 0 0 0 1 1 0 1]
 [0 0 0 0 0 0 0 0]
 [2 0 0 0 0 1 1 0]
 [4 0 2 0 1 0 1 4]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0]
 [1 0 2 0 1 1 1 1]
 [0 0 0 0 0 2 0 0]
 [0 0 1 0 1 1 2 0]
 [0 1 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 1]
 [0 0 1 1 0 3 1 0]
 [0 0 1 0 0 2 1 1]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0]
 [2 0 1 0 1 1 1 2]
 [0 0 1 0 0 0 1 1]]
[0.5  0.15 0.45 0.05 0.3  0.75 0.45 0.6 ]


In [8]:
from collections import Counter
bitstrings = [tuple(i) for i in results.samples]
counts = {k:v for k, v in Counter(bitstrings).items()}
print(counts[(0, 0, 0, 0, 0, 0, 0, 0)])


4


In [9]:
job = eng.run_async(prog, shots=100)

print(job.id)
print(job.status)


ba55ac60-6cfd-43ed-a188-66d2667f3bf6
open


In [15]:
job.clear()
job.status

'complete'

In [16]:

job.wait()
print(job.status)


complete


In [17]:
result = sf.Result(job.result)
print(result.samples.shape)


(100, 8)


In [18]:
prog_compiled = prog.compile(device=eng.device)
prog_compiled.print()


S2gate(0, 0) | (q[2], q[6])
S2gate(1, 0) | (q[0], q[4])
S2gate(1, 0) | (q[1], q[5])
S2gate(1, 0) | (q[3], q[7])
MZgate(0.919, 3.214) | (q[0], q[1])
MZgate(0.9459, 5.946) | (q[2], q[3])
MZgate(1.274, 5.906) | (q[1], q[2])
MZgate(0.1612, 3.33) | (q[0], q[1])
MZgate(1.987, 0.4934) | (q[2], q[3])
MZgate(0.4917, 0.7384) | (q[1], q[2])
Rgate(1.738) | (q[0])
Rgate(3.34) | (q[1])
Rgate(1.357) | (q[2])
Rgate(5.606) | (q[3])
MZgate(0.919, 3.214) | (q[4], q[5])
MZgate(0.9459, 5.946) | (q[6], q[7])
MZgate(1.274, 5.906) | (q[5], q[6])
MZgate(0.1612, 3.33) | (q[4], q[5])
MZgate(1.987, 0.4934) | (q[6], q[7])
MZgate(0.4917, 0.7384) | (q[5], q[6])
Rgate(1.738) | (q[4])
Rgate(3.34) | (q[5])
Rgate(1.357) | (q[6])
Rgate(5.606) | (q[7])
MeasureFock | (q[0], q[1], q[2], q[3], q[4], q[5], q[6], q[7])
